In [2]:
import re
import random
import time
import black
import os
import yaml
import pathlib
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.chrome import ChromeDriverManager

from utils.poppup import popupmsg
from utils.configs import get_config
import selenium

In [3]:
class Driver:
    def __init__(self, port=2023, chromedriver_path=None):
        self.port = port
        self.chromedriver_path = chromedriver_path
        self.driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

In [4]:
class Website:

    def __init__(self, name, url, target_pattern, absolute_url, title_tag, ean_tag, price_tag, seller_tag, captcha_tag, seller_listing_tag):
        self.name = name
        self.url = url
        self.target_pattern = target_pattern
        self.absolute_url = absolute_url
        self.title_tag = title_tag
        self.ean_tag = ean_tag
        self.price_tag = price_tag
        self.seller_tag = seller_tag
        self.captcha_tag = captcha_tag
        self.seller_listing_tag = seller_listing_tag

In [5]:
class Content:

    def __init__(self, timestemp, url, title, ean, infos: dict):
        self.timestemp = timestemp
        self.url = url
        self.title = title
        self.ean = ean
        self.infos = infos

=====================

In [7]:
def get_info_site(site="cdiscount"):
    site = site
    for key,val in get_config()["sites"].items():
        print(key)
        if key == site:
            return val

info_site = get_info_site()

cdiscount


In [8]:
cdiscound = Website(name = info_site.get("name"), 
                    url = info_site.get("url"), 
                    target_pattern = info_site.get("target_pattern"), 
                    absolute_url = info_site.get("absolute_url"), 
                    title_tag = info_site.get("title_tag"), 
                    ean_tag = info_site.get("ean_tag"), 
                    price_tag = info_site.get("price_tag"), 
                    seller_tag = info_site.get("seller_tag"), 
                    captcha_tag = info_site.get("captcha_tag"), 
                    seller_listing_tag = info_site.get("seller_listing_tag") )

In [11]:
#os.listdir(os.getcwd()+"/Téléchargements/chromedriver_linux64/LICENSE.chromedriver")

In [12]:
chromedriver_path = os.getcwd()+"/Téléchargements/chromedriver_linux64"+"/chromedriver.exe"

In [44]:
class Crawler:
    def __init__(self, website_info):
        self.website_info = website_info
        #self.site = site
        # TODO: save visited pages to cloud storage
        self.visited = set()

    def get_page(self, url):
        driver = Driver()
        try:
            driver.driver.get(url)
        except Exception as e:
            msg = f"This exception: {e} was raise when we try to get the page: {url}"
            popupmsg(msg)
            return None
        return BeautifulSoup(driver.driver.page_source, 'html.parser')

    def wait(delay_min, delay_max):
        random_delay = random.randint(delay_min, delay_max)
        print(f"We wait for {random_delay} 's before continue")
        time.sleep(random_delay)

    def safe_get(self, pageObj, selector):
        selectedElems = pageObj.select(selector)
        if selectedElems is not None and len(selectedElems) > 0:
            return '\n'.join([elem.get_text() for elem in selectedElems])
        return ''

    def check_captcha(self, pageObj):
        is_captchat_found = False 
        try:
            selected_elems = pageObj.select(self.website_info.captcha_tag)
            is_captchat_found = selected_elems is not None and len(selected_elems) > 0
        except:
            is_captchat_found = False
        return is_captchat_found

    def parse(self, url):
        bs = self.get_page(url)
        if bs is not None:
            title = self.safe_get(bs, self.website_info.title_tag)
            body = self.safe_get(bs, self.website_info.body_tag)
            if title != '' and body != '':
                content = Content(url, title, body)
                content.print()

    def crawl(self,url):
        """
        Get pages from website home page
        """
        bs = self.get_page(url)
        print(re.compile(self.website_info.target_pattern))
        targetPages = bs.findAll('a', href=re.compile(self.website_info.target_pattern))
        for targetPage in targetPages:
            targetPage = targetPage.attrs['href']
            if targetPage not in self.visited:
                self.visited.append(targetPage)
                if not self.website_info.absoluteUrl:
                    targetPage = '{}{}'.format(self.website_info.url, targetPage)
                #self.parse(targetPage)

In [45]:
crawler = Crawler(cdiscound)
crawler.crawl("https://www.cdiscount.com/informatique/ordinateurs-pc-portables/apple-14-macbook-pro-2023-puce-apple-m2-pro/f-107096402-mphe3fna.html#mpos=0|cd")

re.compile('(\\/f-\\d+.*\\.html)|(\\/mpv.*-\\d+.*\\.html)')
